## HW4: Simple Simulation of Alanine Dipeptide Tutorial
##### Amber = Assisted Model Building and Energy Refinement

#### https://ambermd.org/tutorials/basic/tutorial0/index.php

$ srun -c4 -t2:00:00 --mem=4000 --pty /bin/bash 

$ module load amber/openmpi/intel/22.03

#### Part 1: Prepare topology and coordinate files

In [ ]:
# Start tLEaP
# tLEap is used for preparing and manipulating molecular systems

$ tleap

In [ ]:
# Load the FF19SB force field
# Generate a leap.log file

$ source leaprc.protein.ff19SB

In [ ]:
# Build alanine dipeptide
# Creates a dipeptide named diala with the sequence ACE-ALA-NME

$ diala = sequence { ACE ALA NME }

In [ ]:
# Load the parameters for the OPC (Optimized Potential for Liquid Simulations) water model
# The leaprc.water.opc file contains the necessary parameters for using OPC water
# The FF19SB force field gives the best performance with the OPC water model

$ source leaprc.water.opc

In [ ]:
# Solvates the diala dipeptide in an octahedral box of OPC water.
# solvateoct command is used to create an octahedral box, 
# diala is the molecule to be solvated
# OPCBOX specifies the type of water to use (in this case, OPC water)
# 10.0 is the buffer distance in angstroms between the solute and the edge of the water box.

$ solvateoct diala OPCBOX  10.0

In [ ]:
# Save the Amber parm7 and rst7 input files
# parm7 - describes parameter and topology of the molecules in the system
# rst7 - describes the initial molecular coordinates of the system
# mdin - describes the settings for the Amber MD engine

$ saveamberparm diala parm7 rst7

#### Part 2: Prepare Amber MD pmemd and sander input files

In [ ]:
# Create the file 01_Min.in that includes the following settings for minimization
    """
Minimize
 &cntrl
  imin=1,
  ntx=1,
  irest=0,
  maxcyc=2000,
  ncyc=1000,
  ntpr=100,
  ntwx=0,
  cut=8.0,
 /
    """

In [ ]:
# Create the file 02_Heat.in that includes the following settings for heating
    """
Heat
 &cntrl
  imin=0,
  ntx=1,
  irest=0,
  nstlim=10000,
  dt=0.002,
  ntf=2,
  ntc=2,
  tempi=0.0,
  temp0=300.0,
  ntpr=100,
  ntwx=100,
  cut=8.0,
  ntb=1,
  ntp=0,
  ntt=3,
  gamma_ln=2.0,
  nmropt=1,
  ig=-1,
 /
&wt type='TEMP0', istep1=0, istep2=9000, value1=0.0, value2=300.0 /
&wt type='TEMP0', istep1=9001, istep2=10000, value1=300.0, value2=300.0 /
&wt type='END' /
    """

In [ ]:
# Create the file 03_Prod.in with the settings for production MD
    """
Production
 &cntrl
  imin=0,
  ntx=5,
  irest=1,
  nstlim=5000000,
  dt=0.002,
  ntf=2,
  ntc=2,
  temp0=300.0,
  ntpr=100,
  ntwx=100,
  cut=8.0,
  ntb=2,
  ntp=1,
  ntt=3,
  barostat=1,
  gamma_ln=2.0,
  ig=-1,
 /
    """

### Part 3: Run Amber MD pmemd

In [ ]:
# Check the location of the AMBER_HOME on Greene

$ module show amber/openmpi/intel/22.03

# setenv("AMBER_HOME","/share/apps/amber/22.03/openmpi/intel")

In [ ]:
# Run minimization

$ /share/apps/amber/22.03/openmpi/intel/bin/sander -O -i 01_Min.in -o 01_Min.out -p parm7 -c rst7 -r 01_Min.ncrst \-inf 01_Min.mdinfo

In [ ]:
# Run heating MD

$ /share/apps/amber/22.03/openmpi/intel/bin/sander -O -i 02_Heat.in -o 02_Heat.out -p parm7 -c 01_Min.ncrst \-r 02_Heat.ncrst -x 02_Heat.nc -inf 02_Heat.mdinfo

In [ ]:
# Run the production MD of alanine dipeptide with pmemd
# With the "&" at the end of the command, sander now runs in the background

$ /share/apps/amber/22.03/openmpi/intel/bin/pmemd -O -i 03_Prod.in -o 03_Prod.out -p parm7 -c 02_Heat.ncrst \-r 03_Prod.ncrst -x 03_Prod.nc -inf 03_Prod.info &

# monitor the status of this job use the program tail to print the output file to the terminal [~ 13 hours]
$ tail -f 03_Prod.out